# Analyze Data

In [ ]:
install.packages('Kendall')

In [ ]:
library(arrow, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(ggplot2)
library(tools)
library(tidyr)
library(stringr)
library(purrr)
library(gridExtra)
library(glue)
library(scales)
library(Kendall)
library(lmtest)

In [ ]:
source(here::here("R", "theme_truveta.r"), chdir = TRUE)
source(here::here("R", "write_as_xtable.r"), chdir = TRUE)

In [ ]:
results_dir <- here::here("results")
dir.create(results_dir, recursive = TRUE, showWarnings = FALSE)
data_dir <- here::here("data")
dir.create(data_dir, recursive = TRUE, showWarnings = FALSE)

## Load Data

In [ ]:
df <- arrow::read_parquet(file.path(data_dir, "feature_table.parquet"))
#df |> head(5)

In [ ]:
# Adjust labels for tables: group the less common drugs together & make prettier labels
df <- df |>
    dplyr::mutate(
        group_short = factor(ifelse(as.character(group) %in% c('Exenatide','Lixisenatide'), 'Other', as.character(group)), 
                        levels = c('Semaglutide', 'Dulaglutide', 'Liraglutide', 'Tirzepatide', 'Other'),
                        labels = toTitleCase(c('Semaglutide', 'Dulaglutide', 'Liraglutide', 'Tirzepatide', 'Other'))),
        year = as.factor(IndexYear),
        approved_condition = factor(approved_condition, 
                    levels = c("t2d","obesity","unknown"),
                    labels = c("T2D", "Obesity", "Unknown"))
    )

In [ ]:
# Replace long names with shorter ones
df <- df |>
    dplyr::mutate(
        Race = dplyr::recode(Race, `Black or African American` = "Black"),
        Race = dplyr::recode(Race, `American Indian or Alaska Native` = "AI or AN"),
        Race = dplyr::recode(Race, `Native Hawaiian or Other Pacific Islander` = "NH or PI"),
    )


# Summary Values
This section creates a list of values to export for use in Overleaf.

In [ ]:
summary_values <- list()

In [ ]:
format_pct <- function(x){
    round(x*100,1)
    }

In [ ]:
# N
summary_values$n <- nrow(df)

# n & p with T2D
summary_values$n_t2d <- nrow(df[df$HadT2DEvidence,])
summary_values$p_t2d <- format_pct(mean(df$HadT2DEvidence))

# n & p with overweight or obesity
summary_values$n_ovob <- nrow(df[df$HadOverweightOrObesity,])
summary_values$p_ovob <- format_pct(mean(df$HadOverweightOrObesity))

In [ ]:
# Demographics

# Female
summary_values$n_female <- nrow(df[df$Sex == "Female",])
summary_values$p_female <- format_pct(mean(df$Sex == "Female"))

# Age
summary_values$mean_age <- round(mean(df$AgeInYears),1)
summary_values$sd_age <- round(sd(df$AgeInYears),1)
summary_values$p_4560 <- format_pct(mean(df$AgeGroup == "45-64"))

# Race
summary_values$p_white <- format_pct(mean(df$Race == "White"))
summary_values$p_BlackAA <- format_pct(mean(df$Race == "Black*"))

# A1c Availability
summary_values$p_hasA1c =  format_pct(mean(!is.na(df$a1c)))

In [ ]:
# Time period comparisons
janjun18 <- df |> filter(Period == "2018 Jan-Jun")
janjun23 <- df |> filter(Period == "2023 Jan-Jun")

summary_values$n_janjun_18 <-nrow(janjun18)
summary_values$n_janjun_23 <-nrow(janjun23)

summary_values$p_t2d_janjun_18 <-format_pct(mean(janjun18$HadT2DEvidence))
summary_values$p_ovob_janjun_18 <-format_pct(mean(janjun18$HadOverweightOrObesity))

summary_values$p_t2d_janjun_23 <-format_pct(mean(janjun23$HadT2DEvidence))
summary_values$p_ovob_janjun_23 <- format_pct(mean(janjun23$HadOverweightOrObesity))

# summary_values

In [ ]:
# Tests for differences in baseline characteristics in Jan-Jun 2018 vs. 2023

# Is proportion with T2D in 2018 greater than the proportion in 2023?
summary_values$t2d_18v23_pval <- prop.test(x = c(
                sum(df$Period == "2018 Jan-Jun" & df$HadT2DEvidence), 
                sum(df$Period == "2023 Jan-Jun" & df$HadT2DEvidence)),
          n = c(
               sum(df$Period == "2018 Jan-Jun"), 
                sum(df$Period == "2023 Jan-Jun")),
          alternative = "greater") |> 
    broom::tidy()|>
    pull(p.value) 

# Is proportion with oveweight or obesity in 2018 less than the proportion in 2023?
summary_values$ovob_18v23_pval <- prop.test(x = c(
                sum(df$Period == "2018 Jan-Jun" & df$HadOverweightOrObesity), 
                sum(df$Period == "2023 Jan-Jun" & df$HadOverweightOrObesity)),
          n = c(
               sum(df$Period == "2018 Jan-Jun"), 
                sum(df$Period == "2023 Jan-Jun")),
          alternative = "less") |> 
    broom::tidy() |>
   pull(p.value) 


# Is proportion with a non-missing A1c in 2018 greater than the proportion in 2023?
summary_values$a1cmiss_18v23_pval <-prop.test(x = c(
                sum(df$Period == "2018 Jan-Jun" & !is.na(df$a1c)), 
                sum(df$Period == "2023 Jan-Jun" & !is.na(df$a1c))),
          n = c(
               sum(df$Period == "2018 Jan-Jun"), 
                sum(df$Period == "2023 Jan-Jun")),
          alternative = "greater") |> 
    broom::tidy() |>
   pull(p.value) 


# Are mean A1C values 2018 greater than in 2023?
summary_values$a1c_18v23_pval <- 
    t.test(df[df$Period == '2018 Jan-Jun' & !is.na(df$a1c),]$a1c, 
           df[df$Period == '2023 Jan-Jun' & !is.na(df$a1c),]$a1c, 
           alternative = "greater") |>
    broom::tidy()|>
    pull(p.value) 


# Is proportion with any ADM use in 2018 greater than the proportion in 2023?
df <- df |> 
    mutate(any_adm = pmax(UsedMetformin, UsedSGLT2, UsedDPP4, UsedSulfonylurea, UsedInsulin))
           
# Test whether proportion with T2D in 2018 is less than in 2023 
summary_values$adm_use_18v23_pval <-
    prop.test(x = c(
                sum(df$Period == "2018 Jan-Jun" & df$any_adm == 1), 
                sum(df$Period == "2023 Jan-Jun" & df$any_adm == 1)),
          n = c(
               sum(df$Period == "2018 Jan-Jun"), 
                sum(df$Period == "2023 Jan-Jun")),
    alternative = "greater") |> 
    broom::tidy() |>
   pull(p.value) 

#summary_values

In [ ]:
# For specific medication

# Semaglutide
summary_values$n_sem = sum(df$group == "Semaglutide")
summary_values$p_sem = format_pct(mean(df$group == "Semaglutide"))

# Semaglutide in last time period
summary_values$n_semaglutide_janjun23 <- sum(janjun23$group == "Semaglutide")
summary_values$p_semaglutide_janjun23 <- format_pct(mean(janjun23$group == "Semaglutide"))

# Branded semaglutide
sem_branded <- df |> filter(group == "Semaglutide" & brand_inferred != "Unknown")
summary_values$n_brand_sem = nrow(sem_branded)
summary_values$p_brand_sem_ozempic = format_pct(mean(sem_branded$brand_inferred == "Ozempic"))
summary_values$p_brand_sem_rybelsus = format_pct(mean(sem_branded$brand_inferred == "Rybelsus"))
summary_values$p_brand_sem_wegovy =format_pct(mean(sem_branded$brand_inferred == "Wegovy"))

# Tirzepatide Tirzepatide vs everything else
summary_values$n_tirzepatide = sum(df$group == "Tirzepatide")
summary_values$p_tirzepatide = format_pct(mean(df$group == "Tirzepatide"))

# Demographics - tirzepatide vs everything else
summary_values$p_tirzepatide_female =  format_pct(mean(df[df$group == "Tirzepatide",]$Sex == "Female"))
summary_values$p_nottirzepatide_female =  format_pct(mean(df[df$group != "Tirzepatide",]$Sex == "Female"))
summary_values$mean_age_tirzepatide  =  round(mean(df[df$group == "Tirzepatide",]$AgeInYears),1)
summary_values$mean_age__nottirzepatide =  round(mean(df[df$group != "Tirzepatide",]$AgeInYears, na.rm = TRUE), 1)

summary_values$p_tirzepatide_t2d =  format_pct(mean(df[df$group == "Tirzepatide",]$HadT2DEvidence))
summary_values$p_tirzepatide_hasA1c =  format_pct(mean(!is.na(df[df$group == "Tirzepatide",]$a1c)))
summary_values$p_nottirzepatide_hasA1c =  format_pct(mean(!is.na(df[df$group != "Tirzepatide",]$a1c)))

# A1Cs for Tirzepatide vs everything else
summary_values$mean_a1c_tirzepatide  =  round(mean(df[df$group == "Tirzepatide",]$a1c, na.rm = TRUE),1)
summary_values$mean_a1c__nottirzepatide =  round(mean(df[df$group != "Tirzepatide",]$a1c, na.rm = TRUE), 1)
summary_values$sd_a1c_tirzepatide  =  round(sd(df[df$group == "Tirzepatide",]$a1c, na.rm = TRUE),1)
summary_values$sd_a1c__nottirzepatide =  round(sd(df[df$group != "Tirzepatide",]$a1c, na.rm = TRUE), 1)

#summary_values

In [ ]:
# Tests by specific medication 
# Is the mean age for patients on Tirzepatide lower than for other medications?
summary_values$tirzepatide_agediff_pval <- 
    t.test(
          df[df$group == "Tirzepatide",]$AgeInYears,
          df[df$group != "Tirzepatide",]$AgeInYears, 
          alternative = "less") |>
    broom::tidy() |>
   pull(p.value) 

# Is the proportion of patients who are female greater for Tirzepatide vs other medications
summary_values$tirzepatide_female_pval <-
    prop.test(x = c(
              sum(df$group == "Tirzepatide" & df$Sex == "Female"),             
              sum(df$group != "Tirzepatide" & df$Sex == "Female")),
          n = c(
              sum(df$group == "Tirzepatide"),  
              sum(df$group != "Tirzepatide")),
          alternative = "greater") |> 
    broom::tidy() |>
    pull(p.value)


# Is the proportion of patients who have an A1C value lower for Tirzepatide vs other medications
summary_values$tirzepatide_hasa1c_pval <-
    prop.test(x = c(
              sum(df$group == "Tirzepatide" & !is.na(df$a1c)),             
              sum(df$group != "Tirzepatide" & !is.na(df$a1c))),
          n = c(
              sum(df$group == "Tirzepatide"),  
              sum(df$group != "Tirzepatide")),
          alternative = "less") |> 
    broom::tidy() |>
    pull(p.value)


# Are mean A1C values lower for tirzepatide than other GLP1 RAs?
summary_values$tirzepatide_a1cdiff_pval <- 
    t.test(df[df$group == "Tirzepatide" & !is.na(df$a1c),]$a1c, 
           df[df$group != "Tirzepatide" & !is.na(df$a1c),]$a1c, 
           alternative = "less") |>
    broom::tidy()|>
    pull(p.value) 


summary_values

In [ ]:
# Brands
# Female proportion by brand
summary_values$p_wegovy_female =  format_pct(mean(df[df$brand_inferred == "Wegovy",]$Sex == "Female"))
summary_values$p_ozempic_female  =  format_pct(mean(df[df$brand_inferred == "Ozempic",]$Sex == "Female"))
summary_values$p_rybelsus_female  =  format_pct(mean(df[df$brand_inferred == "Rybelsus",]$Sex == "Female"))

# Age by brand
summary_values$mean_age_wegovy =  round(mean(df[df$brand_inferred == "Wegovy",]$AgeInYears),1)
summary_values$mean_age_ozempic  =  round(mean(df[df$brand_inferred == "Ozempic",]$AgeInYears),1)
summary_values$mean_age_rybelsus  =  round(mean(df[df$brand_inferred == "Rybelsus",]$AgeInYears),1)

In [ ]:
# Tests by brand
# Is the mean age for patients on Wegovy lower than those on Ozempic?
summary_values$Weg_vOz_age_pval <- 
    t.test(
          df[df$brand_inferred == "Wegovy",]$AgeInYears,
          df[df$brand_inferred == "Ozempic",]$AgeInYears, 
          alternative = "less") |>
    broom::tidy() |>
   pull(p.value) 

# Is the mean age for patients on Wegovy lower than those on Rybelsus?
summary_values$Weg_vRy_age_pval <- 
    t.test(
          df[df$brand_inferred == "Wegovy",]$AgeInYears,
          df[df$brand_inferred == "Rybelsus",]$AgeInYears, 
          alternative = "less") |>
    broom::tidy() |>
   pull(p.value) 

# Is the proportion of patients who are female greater for Wegovy compared to Ozempic?
summary_values$Weg_vOz_female_pval <-
    prop.test(x = c(
              sum(df$brand_inferred == "Wegovy" & df$Sex == "Female"),             
              sum(df$brand_inferred == "Ozempic" & df$Sex == "Female")),
          n = c(
              sum(df$brand_inferred == "Wegovy"),  
              sum(df$brand_inferred == "Ozempic")),
          alternative = "greater") |> 
    broom::tidy() |>
    pull(p.value)


# Is the proportion of patients who are female greater for Wegovy compared to Rybelsus?
summary_values$Weg_vRy_female_pval <-
    prop.test(x = c(
              sum(df$brand_inferred == "Wegovy" & df$Sex == "Female"),             
              sum(df$brand_inferred == "Rybelsus" & df$Sex == "Female")),
          n = c(
              sum(df$brand_inferred == "Wegovy"),  
              sum(df$brand_inferred == "Rybelsus")),
          alternative = "greater") |> 
    broom::tidy() |>
    pull(p.value)


summary_values

In [ ]:
# labelled Use
summary_values$n_known_label = sum(df$approved_condition!="Unknown")
summary_values$p_known_label = format_pct(mean(df$approved_condition!="Unknown"))
summary_values$n_known_label_t2d = sum(df$approved_condition=="T2D")
summary_values$n_known_label_ovob =  sum(df$approved_condition=="Obesity")

# On-label for T2D 
summary_values$n_known_label_t2d_onlabel =  sum(df$approved_condition=="T2D" & df$HadT2DEvidence)
summary_values$p_known_label_t2d_onlabel =  format_pct(sum(df$approved_condition=="T2D" & df$HadT2DEvidence)/sum(df$approved_condition=="T2D"))

summary_values$p_t2d_onlabel_janjun18 =  format_pct(sum(df$approved_condition=="T2D" & df$HadT2DEvidence & df$Period == "2018 Jan-Jun")/ sum(df$approved_condition=="T2D" & df$Period == "2018 Jan-Jun"))
summary_values$p_t2d_onlabel_janjun23 =  format_pct(sum(df$approved_condition=="T2D" & df$HadT2DEvidence & df$Period == "2023 Jan-Jun")/ sum(df$approved_condition=="T2D" & df$Period == "2023 Jan-Jun"))

# On-label for overweight or obesity
summary_values$n_known_label_ovob_onlabel =  sum(df$approved_condition=="Obesity" & df$HadOverweightOrObesity)
summary_values$p_known_label_ovob_onlabel =  format_pct(sum(df$approved_condition=="Obesity" & df$HadOverweightOrObesity)/ sum(df$approved_condition=="Obesity"))

summary_values$p_oo_onlabel_janjun18 =  format_pct(sum(df$approved_condition=="Obesity" & df$HadOverweightOrObesity & df$Period == "2018 Jan-Jun")/ sum(df$approved_condition=="Obesity" & df$Period == "2018 Jan-Jun"))
summary_values$p_oo_onlabel_janjun23 =  format_pct(sum(df$approved_condition=="Obesity" & df$HadOverweightOrObesity & df$Period == "2023 Jan-Jun")/ sum(df$approved_condition=="Obesity" & df$Period == "2023 Jan-Jun"))

#summary_values

In [ ]:
# Tests for differences by labelled use
# Is proportion of T2D-labeled prescriptions that are on-label greater in 2018 than 2023?
summary_values$t2d_onlabel_18v23_pval <-prop.test(x = c(
                sum(df$Period == "2018 Jan-Jun" & df$approved_condition == "T2D" & df$HadT2DEvidence), 
                sum(df$Period == "2023 Jan-Jun"  & df$approved_condition == "T2D"& df$HadT2DEvidence)),
          n = c(
                sum(df$Period == "2018 Jan-Jun" & df$approved_condition == "T2D"), 
                sum(df$Period == "2023 Jan-Jun"  & df$approved_condition == "T2D")),
          alternative = "greater") |> 
    broom::tidy() |>
   pull(p.value) 

known_label <-df[df$approved_condition!="Unknown",]

# Characteristics 
summary_values$ovoblabel_p_female <- format_pct(mean(known_label[known_label$approved_condition == "Obesity",]$Sex == "Female"))
summary_values$ovoblabel_p_mean_age <- round(mean(known_label[known_label$approved_condition == "Obesity",]$AgeInYears),1)
summary_values$ovoblabel_p_sd_age <- round(sd(known_label[known_label$approved_condition == "Obesity",]$AgeInYears),1)


summary_values$t2dlabel_p_female <- format_pct(mean(known_label[known_label$approved_condition == "T2D",]$Sex == "Female"))
summary_values$t2dlabel_p_mean_age <- round(mean(known_label[known_label$approved_condition == "T2D",]$AgeInYears),1)
summary_values$t2dlabel_p_sd_age <- round(sd(known_label[known_label$approved_condition == "T2D",]$AgeInYears),1)


# Is the mean age for patients on obesity-labeled medications lower than for T2D-labeled medications?
summary_values$labeluse_agediff_pval <- 
    t.test(
          known_label[known_label$approved_condition == "Obesity",]$AgeInYears,
          known_label[known_label$approved_condition == "T2D",]$AgeInYears, 
          alternative = "less") |>
    broom::tidy() |>
   pull(p.value) 

# Is the proportion of patients who are female higher for obesity-labeled medications lower than for T2D-labeled medications?
summary_values$labeluse_female_pval <-
    prop.test(x = c(
              sum(known_label$approved_condition == "Obesity" & known_label$Sex == "Female"),             
              sum(known_label$approved_condition == "T2D" & known_label$Sex == "Female")),
          n = c(
              sum(known_label$approved_condition == "Obesity"),  
              sum(known_label$approved_condition == "T2D")),
          alternative = "greater") |> 
    broom::tidy() |>
    pull(p.value)

summary_values

In [ ]:
# How many states with greater than XX threshold patients?
state_pt_threshold  = 1000 # decide a threshold of number of patients at which to count a state 

state_total <- df |> group_by(LatestStateOrProvince) |> tally() |> arrange(desc(n)) |> filter(!is.na(LatestStateOrProvince))
summary_values$n_states <-  n_distinct(state_total[state_total$n >= state_pt_threshold,]$LatestStateOrProvince)
summary_values$state_pt_threshold <- state_pt_threshold

In [ ]:
## Monthly by brand - which is most common and what % is it?
month_totals <- df |> group_by(IndexYearMonth) |> tally() |> rename("month_total" = n)

semaglutide_most_common <- df |> 
group_by(group, IndexYearMonth) |> 
tally() |> 
ungroup() |>
inner_join(month_totals, by = "IndexYearMonth") |>
mutate(p = n/month_total) |>
arrange(IndexYearMonth, desc(n))|>
group_by(IndexYearMonth) |>
slice(1) |>
ungroup() |> 
arrange(desc(IndexYearMonth)) |> 
mutate(same_as_previous = replace_na(lag(group) == group, TRUE),
      continuous_months_most_common = row_number() == cumsum(same_as_previous)) |> 
filter(group == "Semaglutide" & continuous_months_most_common)

# First month where semaglutide was most common
first_m <- semaglutide_most_common |> slice_min(IndexYearMonth) |> pull(IndexYearMonth)
first_m

# Last month where semaglutide was most common
last_m <- semaglutide_most_common |> slice_max(IndexYearMonth)|> pull(IndexYearMonth)
last_m

# Total during this period 
summary_values$semaglutide_most_common_n <- df |>  filter(IndexYearMonth >= first_m & IndexYearMonth <= last_m) |> 
        summarize(n = sum(group == "Semaglutide")) |>
        pull(n)
                                                                                 

summary_values$semaglutide_most_common_p <- df |>  filter(IndexYearMonth >= first_m & IndexYearMonth <= last_m) |> 
        summarize(p = format_pct(mean(group == "Semaglutide"))) |>
        pull(p)


summary_values

# Models

In [ ]:
# Total volume per month 
total <- df |> 
    arrange(IndexYearMonth) |> 
    group_by(IndexYearMonth) |> 
    tally() 

In [ ]:
# Seasonally adjust 
ts_total <- ts( total |>  select(-IndexYearMonth),frequency = 12)

# Decompose into trend vs seasonal
decomp <- decompose(ts_total, "additive")
plot(decomp)
seasonal_comp <- decomp$seasonal
trend_comp <- decomp$trend

ts_seasonally_adjusted = ts_total -decomp$seasonal
plot(ts_seasonally_adjusted)

In [ ]:
# AR1 Model 
ts_sa = as.data.frame(cbind(total,ts_seasonally_adjusted)) |> 
    rename(n_sa = ts_total) |> 
    mutate(t = row_number(),
           march2020 = (IndexYearMonth == lubridate::as_date('2020-03-01')))

# Non-parametric Kendall Trend Test
Kendall::Kendall(ts_sa$IndexYearMonth, ts_sa$n_sa)

# Do we need to include an indicator for March 2020? (where we expect the impact of the relationship to lag is substantially different than all other timepoints)
test_modfull <-  lm(n_sa ~ lag(n_sa) + march2020 , data = ts_sa) 
test_modnest <-  lm(n_sa ~ lag(n_sa) , data = ts_sa) 
lrtest(test_modfull, test_modnest)
# Not needed

# AR1 Model 
ar1 <- lm(n_sa ~ lag(n_sa), data = ts_sa) 
broom::tidy(ar1)
summary_values$trend_pval = broom::tidy(ar1) |> filter(term == "lag(n_sa)") |> pull(p.value)

# Linear regression (to compare)
lm(n_sa ~ t, data = ts_sa) |> broom::tidy(conf.int = TRUE) 

In [ ]:
get_ar1 <- function(data){
    dat_total <- data |> group_by(IndexYearMonth) |> arrange(IndexYearMonth) |> tally() |> ungroup()

    dat_total_ts <- ts(dat_total |>  select(-IndexYearMonth),frequency = 12)
    
    dc <- decompose(dat_total_ts, "additive")
    tsa = dat_total_ts -dc$seasonal


    ts_sa = as.data.frame(cbind(dat_total,tsa)) |> 
        rename(n_sa = dat_total_ts) |> 
        mutate(t = row_number())

    # AR1 Model 
    ar1mod <- lm(n_sa ~ lag(n_sa), data = ts_sa) 
    return(ar1mod)
   }


In [ ]:
# Semaglutide
get_ar1(df[df$group == "Semaglutide",]) |> broom::tidy()

# Liraglutide
get_ar1(df[df$group == "Liraglutide",]) |> broom::tidy()

# Tirzepatide
## Tirzepatide can't be decomposed because it's too new -- most months only have one observation. We will fit the AR model on the raw data
    tirzepatide_total <- df |> 
        filter(group == "Tirzepatide" & IndexYearMonth >= as.Date("2022-06-01")) |> 
        group_by(IndexYearMonth) |> 
        arrange(IndexYearMonth) |>
        tally() |> 
        ungroup()

lm(n ~ lag(n), data = tirzepatide_total) |> broom::tidy()

In [ ]:
save(summary_values, file = file.path(results_dir, 'summary_values.rdata'))

# Table 1s

In [ ]:
# Labels
table1::label(df$AgeGroup) <- "Age Group"
table1::label(df$AgeInYears) <- "Age"
                 
table1::label(df$HadT2DEvidence) <- "T2D"
table1::label(df$HadOverweightOrObesity) <- "Obesity or Overweight"
table1::label(df$year) <- "Year"
                 
table1::label(df$group) <- "Generic"
table1::label(df$brand_inferred) <- "Brand"
table1::label(df$approved_condition) <- "FDA-labeled Use"

table1::label(df$HadAFib) <- "Atrial Fibrilation"
table1::label(df$HadAsthma) <- "Asthma"
table1::label(df$HadCKD) <- "CKD"
table1::label(df$HadCOPD) <- "COPD"
table1::label(df$HadGlaucoma) <- "Glaucoma"
table1::label(df$HadHF) <- "Heart Failure"
table1::label(df$HadHyperlipidemia) <- "Hyperlipidemia"
table1::label(df$HadHypertension) <- "Hypertension"
table1::label(df$HadIschemicHeartDisease) <- "Ischemic Heart Disease"
table1::label(df$HadPreviousAcuteMI) <- "Acute MI"
table1::label(df$HadPreviousIschemicStroke) <- "Ischemic Stroke"
table1::label(df$HadPreviousMDD) <- "Major Depressive Disorder"
table1::label(df$HadOsteoporosis) <- "Osteoporosis"
table1::label(df$HadPrevBariatricSurgery) <- "Bariatric Surgery"

table1::label(df$UsedMetformin) <- "Metformin"
table1::label(df$UsedSGLT2) <- "SGLT2i"
table1::label(df$UsedDPP4) <- "DPP4"
table1::label(df$UsedSulfonylurea) <- "Sulfonylurea"
table1::label(df$UsedInsulin) <- "Insulin"
table1::label(df$UsedOrlistat) <- "Orlistat"
table1::label(df$UsedPhentermineTopiramate) <- "Phentermine Topiramate"


table1::label(df$weightInPounds) <- "Weight (in lbs)"
table1::label(df$bmi) <- "BMI"
table1::label(df$a1c) <- "HbA1c"


caption_all <- "Abbreviations: 
ADM=anti-diabetic medication,
AI  = American Indian, 
AN = Alaska Native, 
AOM=anti-obesity medication, 
Black = Black or African American, 
BMI = body mass index,
CKD=chronic kidney disease, 
COPD=chronic obstructive pulmonary disease, 
DPP4= dipeptidyl peptidase 4 inhibitor, 
FDA=Food and Drug Administration, 
HbA1c=hemoglobin A1C, 
MI=myocardial infarction, 
NH = Native Hawaiian, 
PI = Pacific Islander, 
SD=standard deviation, 
SGLT2i=sodium/glucose cotransporter-2 inhibitor,
T2D=type 2 diabetes mellitus."

# Render options
rndr <- function(x, ...) {
    y <- table1::render.default(x, ...)
    if (is.logical(x)) y[2] else y
}

my.render.cont <- function(x) {
    with(table1::stats.apply.rounding(table1::stats.default(x), digits=2), c("",
        "Mean (SD)"=sprintf("%s (%s)", MEAN, SD)))
}

my.render.cat <- function (x) 
{
    c("", sapply(table1::stats.default(x), function(y) with(y, 
        sprintf("%d (%0.0f%%)", FREQ, PCT))))
}


### By Generic/Substance

In [ ]:

t1_generic <- table1::table1(
    # Demographics
    ~ AgeInYears + 
    AgeGroup + 
    Sex + 
    Race + 
    Ethnicity  + 
    
    # Indications
    HadT2DEvidence + 
    HadOverweightOrObesity +  +
    approved_condition + 
    
    #Brand
    brand_inferred + 

    #Absolute time of intitation
    #year + 
    Period + # 6 months periods of each year
    
    # Comborbidities
    HadAFib + 
    HadAsthma + 
    HadCKD + 
    HadCOPD + 
    HadGlaucoma + 
    HadHF + 
    HadHyperlipidemia + 
    HadHypertension + 
    HadIschemicHeartDisease + 
    HadPreviousAcuteMI + 
    HadPreviousIschemicStroke + 
    HadPreviousMDD + 
    HadOsteoporosis+ 
    HadPrevBariatricSurgery +
    
    # Previous medications
    UsedMetformin +
    UsedSGLT2 + 
    UsedDPP4 + 
    UsedSulfonylurea +  
    UsedInsulin + 
    UsedOrlistat + 
    UsedPhentermineTopiramate + 
    bmi +
    # Baseline Weight 
    weightInPounds +
    # Baseline A1c 
    a1c
    | group_short,
    data = df,
    render = rndr,
    render.continuous=my.render.cont, 
    render.categorical=my.render.cat
)
IRdisplay::display_html(t1_generic)
write.csv(as.data.frame(t1_generic), file.path(results_dir, "table1_generic.csv"))
t1_genericdf <- as.data.frame(t1_generic)

caption_this_one <-  'Characteristics of patients first prescribed a GLP-1 RA, by specific medication Other specific medication includes the two smallest groups: Exenatide and Lixisenatide.'
caption <- paste0(caption_this_one,"\\", caption_all)
label <- 'tab:table_1_generic'

add.to.row <- list()
add.to.row$pos <- list()
add.to.row$pos[[1]] <- c(0)
add.to.row$command <- 
  paste0(
    "\\hline \n",
    "\\endfirsthead \n",
    "\\multicolumn{7}{p{\\textwidth}}{{ \\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\ \n",
    "\\hline Feature & Semaglutide & Dulaglutide & Liraglutide & Tirzepatide & Other & Overall \\\\ \\hline \n",
    "\\endhead \n",
    "\\hline \\multicolumn{7}{p{\\textwidth}}{{Continued on next page}} \\\\ \\hline \n",
    "\\endfoot \n",
    "\\hline \n",
    "\\endlastfoot \n"
  )

align <- c('', "p{0.2\\textwidth}","p{0.13\\textwidth}", "p{0.13\\textwidth}", "p{0.13\\textwidth}","p{0.13\\textwidth}","p{0.13\\textwidth}","p{0.14\\textwidth}")


xtable::print.xtable(
  xtable::xtable(
    t1_genericdf,
    label = label,
    caption = caption,
    floating = FALSE,
    align = align
  ),
  type = 'latex', 
  file = file.path(results_dir, "table1_generic.tex"), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  floating = FALSE,
  add.to.row = add.to.row,
  tabular.environment = "longtable",
  hline.after = FALSE,
  format.args = list(digits = 1, 
                     big.mark = ",")
)

### By Period
Split into 2018-2020 and 2021-2023 for easier display

In [ ]:
t1_period1 <- table1::table1(
    # Demographics
    ~ AgeInYears + 
    AgeGroup + 
    Sex + 
    Race + 
    Ethnicity  + 
    
    # Indications
    HadT2DEvidence + 
    HadOverweightOrObesity +  +
    approved_condition + 
    
    #Brand
    group + 
    brand_inferred + 

    #Absolute time of intitation
    #year + 
    #period + # 6 months periods of each year
    
    # Comborbidities
    HadAFib + 
    HadAsthma + 
    HadCKD + 
    HadCOPD + 
    HadGlaucoma + 
    HadHF + 
    HadHyperlipidemia + 
    HadHypertension + 
    HadIschemicHeartDisease + 
    HadPreviousAcuteMI + 
    HadPreviousIschemicStroke + 
    HadPreviousMDD + 
    HadOsteoporosis+ 
    HadPrevBariatricSurgery +
    
    # Previous medications
    UsedMetformin +
    UsedSGLT2 + 
    UsedDPP4 + 
    UsedSulfonylurea +  
    UsedInsulin + 
    UsedOrlistat + 
    UsedPhentermineTopiramate + 
    bmi +
    # Baseline Weight 
    weightInPounds +
    # Baseline A1c 
    a1c
    | Period,
    data = df[df$IndexYear %in% c('2018', '2019', '2020'),],
    render = rndr,
    render.continuous=my.render.cont, 
    render.categorical=my.render.cat,
    overall=F

)
IRdisplay::display_html(t1_period1)
write.csv(as.data.frame(t1_period1), file.path(results_dir, "table1_period1.csv"))
t1_period1df <- as.data.frame(t1_period1)

caption_this_one <-  'Characteristics of patients initiating a GLP-1 RA, by period (part 1).'
caption <- paste0(caption_this_one,"\\", caption_all)
label <- 'tab:table_1_period1'

add.to.row <- list()
add.to.row$pos <- list()
add.to.row$pos[[1]] <- c(0)
add.to.row$command <- 
  paste0(
    "\\hline \n",
    "\\endfirsthead \n",
    "\\multicolumn{7}{p{\\textwidth}}{{ \\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\ \n",
    "\\hline Feature & 2018 Jan-Jun & 2018 Jul-Dec & 2019 Jan-Jun & 2019 Jul-Dec & 2020 Jan-Jun & 2020 Jul-Dec \\\\ \\hline \n",
    "\\endhead \n",
    "\\hline \\multicolumn{7}{p{\\textwidth}}{{Continued on next page}} \\\\ \\hline \n",
    "\\endfoot \n",
    "\\hline \n",
    "\\endlastfoot \n"
  )

align <- c('', "p{0.16\\textwidth}","p{0.12\\textwidth}", "p{0.12\\textwidth}", "p{0.12\\textwidth}","p{0.12\\textwidth}","p{0.12\\textwidth}","p{0.12\\textwidth}")

xtable::print.xtable(
  xtable::xtable(
    t1_period1df,
  label = label,
    caption = caption,
    floating = FALSE,
    align = align
  ),
  type = 'latex', 
  file = file.path(results_dir, "table1_period1.tex"), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  floating = FALSE,
  add.to.row = add.to.row,
  tabular.environment = "longtable",
  hline.after = FALSE,
  format.args = list(digits = 3, big.mark = ",")
)

In [ ]:
# Part 2
t1_period2 <- table1::table1(
    # Demographics
    ~ AgeInYears + 
    AgeGroup + 
    Sex + 
    Race + 
    Ethnicity  + 
    
    # Indications
    HadT2DEvidence + 
    HadOverweightOrObesity +  +
    approved_condition + 
    
    #Brand
    group + 
    brand_inferred + 

    #Absolute time of intitation
    #year + 
    #period + # 6 months periods of each year
    
    # Comborbidities
    HadAFib + 
    HadAsthma + 
    HadCKD + 
    HadCOPD + 
    HadGlaucoma + 
    HadHF + 
    HadHyperlipidemia + 
    HadHypertension + 
    HadIschemicHeartDisease + 
    HadPreviousAcuteMI + 
    HadPreviousIschemicStroke + 
    HadPreviousMDD + 
    HadOsteoporosis+ 
    HadPrevBariatricSurgery +
    
    # Previous medications
    UsedMetformin +
    UsedSGLT2 + 
    UsedDPP4 + 
    UsedSulfonylurea +  
    UsedInsulin + 
    UsedOrlistat + 
    UsedPhentermineTopiramate + 
    bmi + 
    # Baseline Weight 
    weightInPounds +
    # Baseline A1c 
    a1c
    | Period,
    data = df[df$IndexYear >2020 ,],
    render = rndr,
    render.continuous=my.render.cont, 
    render.categorical=my.render.cat,
    overall = F

)
#IRdisplay::display_html(t1_period2)
write.csv(as.data.frame(t1_period2), file.path(results_dir, "table1_period2.csv"))
t2_period2df <- as.data.frame(t1_period2)

caption_this_one <-  'Characteristics of patients initiating a GLP-1 RA, by period (part 2).'
caption <- paste0(caption_this_one,"\\", caption_all)

label <- 'tab:table_1_period2'

add.to.row <- list()
add.to.row$pos <- list()
add.to.row$pos[[1]] <- c(0)
add.to.row$command <- 
  paste0(
    "\\hline \n",
    "\\endfirsthead \n",
    "\\multicolumn{6}{p{\\textwidth}}{{ \\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\ \n",
    "\\hline Feature & 2021 Jan-Jun & 2021 Jul-Dec & 2022 Jan-Jun & 2022 Jul-Dec & 2023 Jan-Jun \\\\ \\hline \n",
    "\\endhead \n",
    "\\hline \\multicolumn{6}{p{\\textwidth}}{{Continued on next page}} \\\\ \\hline \n",
    "\\endfoot \n",
    "\\hline \n",
    "\\endlastfoot \n"
  )

align <- c('', "p{0.2\\textwidth}","p{0.12\\textwidth}", "p{0.12\\textwidth}", "p{0.12\\textwidth}","p{0.12\\textwidth}","p{0.12\\textwidth}")


xtable::print.xtable(
  xtable::xtable(
    t2_period2df,
  label = label,
    caption = caption,
    floating = FALSE,
    align = align
  ),
  type = 'latex', 
  file = file.path(results_dir, "table1_period2.tex"), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  floating = FALSE,
  add.to.row = add.to.row,
  tabular.environment = "longtable",
  hline.after = FALSE,
  format.args = list(digits = 3, big.mark = ",")
)

### By Year

In [ ]:
t1_year <- table1::table1(
    # Demographics
    ~ AgeInYears + 
    AgeGroup + 
    Sex + 
    Race + 
    Ethnicity  + 
    
    # Indications
    HadT2DEvidence + 
    HadOverweightOrObesity +  +
    approved_condition + 
    
    #Brand
    group + 
    brand_inferred + 

    #Absolute time of intitation
    #year + 
    #period + # 6 months periods of each year
    
    # Comborbidities
    HadAFib + 
    HadAsthma + 
    HadCKD + 
    HadCOPD + 
    HadGlaucoma + 
    HadHF + 
    HadHyperlipidemia + 
    HadHypertension + 
    HadIschemicHeartDisease + 
    HadPreviousAcuteMI + 
    HadPreviousIschemicStroke + 
    HadPreviousMDD + 
    HadOsteoporosis+ 
    HadPrevBariatricSurgery +
    
    # Previous medications
    UsedMetformin +
    UsedSGLT2 + 
    UsedDPP4 + 
    UsedSulfonylurea +  
    UsedInsulin + 
    UsedOrlistat + 
    UsedPhentermineTopiramate +  
    bmi +
    # Baseline Weight 
    weightInPounds +
    # Baseline A1c 
    a1c
    | year,
    data = df,
    render = rndr,
    render.continuous=my.render.cont, 
    render.categorical=my.render.cat

)
IRdisplay::display_html(t1_year)
write.csv(as.data.frame(t1_year), file.path(results_dir, "table1_year.csv"))
t1_yeardf <- as.data.frame(t1_year)


caption_this_one <-   'Characteristics of patients first prescribed a GLP-1 RA, by year of first prescription.'
caption <- paste0(caption_this_one,"\\", caption_all)

label <- 'tab:table_1_year'




add.to.row <- list()
add.to.row$pos <- list()
add.to.row$pos[[1]] <- c(0)
add.to.row$command <- 
  paste0(
    "\\hline \n",
    "\\endfirsthead \n",
    "\\multicolumn{8}{p{\\textwidth}}{{ \\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\ \n",
    "\\hline Feature & 2018 & 2019 & 2020 & 2021 & 2022 & 2023 & Overall \\\\ \\hline \n",
    "\\endhead \n",
    "\\hline \\multicolumn{8}{p{\\textwidth}}{{Continued on next page}} \\\\ \\hline \n",
    "\\endfoot \n",
    "\\hline \n",
    "\\endlastfoot \n"
  )

align <- c('', "p{0.2\\textwidth}","p{0.1\\textwidth}", "p{0.1\\textwidth}", "p{0.1\\textwidth}","p{0.1\\textwidth}","p{0.1\\textwidth}","p{0.1\\textwidth}", "p{0.1\\textwidth}")


xtable::print.xtable(
  xtable::xtable(
    t1_yeardf,
  label = label,
    caption = caption,
    floating = FALSE,
    align = align
  ),
  type = 'latex', 
  file = file.path(results_dir, "table1_year.tex"), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  floating = FALSE,
  add.to.row = add.to.row,
  tabular.environment = "longtable",
  hline.after = FALSE,
  format.args = list(digits = 3, big.mark = ",")
)

## By Use

In [ ]:
t1_use <- table1::table1(
    # Demographics
    ~ AgeInYears + 
    AgeGroup + 
    Sex + 
    Race + 
    Ethnicity  + 
    
    # Indications
    HadT2DEvidence + 
    HadOverweightOrObesity +  +
    #approved_condition + 
    
    #Brand
    group + 
    brand_inferred + 

    #Absolute time of intitation
    #year + 
    Period + # 6 months periods of each year
    
    # Comborbidities
    HadAFib + 
    HadAsthma + 
    HadCKD + 
    HadCOPD + 
    HadGlaucoma + 
    HadHF + 
    HadHyperlipidemia + 
    HadHypertension + 
    HadIschemicHeartDisease + 
    HadPreviousAcuteMI + 
    HadPreviousIschemicStroke + 
    HadPreviousMDD + 
    HadOsteoporosis+ 
    HadPrevBariatricSurgery +
    
    # Previous medications
    UsedMetformin +
    UsedSGLT2 + 
    UsedDPP4 + 
    UsedSulfonylurea +  
    UsedInsulin + 
    UsedOrlistat + 
    UsedPhentermineTopiramate +  
    bmi +
    # Baseline Weight 
    weightInPounds +
    # Baseline A1c 
    a1c
    | approved_condition,
    data = df,
    render = rndr,
    render.continuous=my.render.cont, 
    render.categorical=my.render.cat

)
IRdisplay::display_html(t1_use)
write.csv(as.data.frame(t1_use), file.path(results_dir, "table1_use.csv"))
t1_usedf <- as.data.frame(t1_use)


caption_this_one <-  'Characteristics of patients first prescribed a GLP-1 RA, by FDA-labeled use.'
caption <- paste0(caption_this_one,"\\", caption_all)

label <- 'tab:table_1_use'

add.to.row <- list()
add.to.row$pos <- list()
add.to.row$pos[[1]] <- c(0)
add.to.row$command <- 
  paste0(
    "\\hline \n",
    "\\endfirsthead \n",
    "\\multicolumn{5}{p{\\textwidth}}{{ \\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\ \n",
    "\\hline Feature & Type II Diabetes & Obesity & Unknown & Overall\\\\ \\hline \n",
    "\\endhead \n",
    "\\hline \\multicolumn{5}{p{\\textwidth}}{{Continued on next page}} \\\\ \\hline \n",
    "\\endfoot \n",
    "\\hline \n",
    "\\endlastfoot \n"
  )

align <- c('', "p{0.2\\textwidth}","p{0.2\\textwidth}", "p{0.2\\textwidth}", "p{0.2\\textwidth}","p{0.2\\textwidth}")


xtable::print.xtable(
  xtable::xtable(
    t1_usedf,
  label = label,
    caption = caption,
    floating = FALSE,
    align = align
  ),
  type = 'latex', 
  file = file.path(results_dir, "table1_use.tex"), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  floating = FALSE,
  add.to.row = add.to.row,
  tabular.environment = "longtable",
  hline.after = FALSE,
  format.args = list(digits = 3, big.mark = ",")
)

In [ ]:
t1_brand1 <- table1::table1(
    # Demographics
    ~ AgeInYears + 
    AgeGroup + 
    Sex + 
    Race + 
    Ethnicity  + 
    
    # Indications
    HadT2DEvidence + 
    HadOverweightOrObesity +  +
    #approved_condition + 
    
    #Brand
    #group + 

    #Absolute time of intitation
    #year + 
    Period + # 6 months periods of each year
    
    # Comborbidities
    HadAFib + 
    HadAsthma + 
    HadCKD + 
    HadCOPD + 
    HadGlaucoma + 
    HadHF + 
    HadHyperlipidemia + 
    HadHypertension + 
    HadIschemicHeartDisease + 
    HadPreviousAcuteMI + 
    HadPreviousIschemicStroke + 
    HadPreviousMDD + 
    HadOsteoporosis+ 
    HadPrevBariatricSurgery +
    
    # Previous medications
    UsedMetformin +
    UsedSGLT2 + 
    UsedDPP4 + 
    UsedSulfonylurea +  
    UsedInsulin + 
    UsedOrlistat + 
    UsedPhentermineTopiramate +  
    bmi +
    # Baseline Weight 
    weightInPounds +
    # Baseline A1c 
    a1c
    | brand_inferred,
    data = df[df$group_short %in% c("Semaglutide", "Liraglutide"),],
    render = rndr,
    render.continuous=my.render.cont, 
    render.categorical=my.render.cat,
    overall = F

)
IRdisplay::display_html(t1_brand1)
write.csv(as.data.frame(t1_brand1), file.path(results_dir, "table1_brand_part1.csv"))
t1_brand1df <- as.data.frame(t1_brand1)

caption_this_one <-  'Characteristics of patients first prescribed a GLP-1 RA, by brand for Semaglutide and Liraglutide.'
caption <- paste0(caption_this_one,"\\", caption_all)
label <- 'tab:table_1_brand_part1'

add.to.row <- list()
add.to.row$pos <- list()
add.to.row$pos[[1]] <- c(0)
add.to.row$command <- 
  paste0(
    "\\hline \n",
    "\\endfirsthead \n",
    "\\multicolumn{7}{p{\\textwidth}}{{ \\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\ \n",
    "\\hline Feature & Ozempic & Rybelsus & Wegovy & Victoza & Saxenda & Unknown\\\\ \\hline \n",
    "\\endhead \n",
    "\\hline \\multicolumn{7}{p{\\textwidth}}{{Continued on next page}} \\\\ \\hline \n",
    "\\endfoot \n",
    "\\hline \n",
    "\\endlastfoot \n"
  )

align <- c('', "p{0.16\\textwidth}","p{0.14\\textwidth}", "p{0.14\\textwidth}", "p{0.14\\textwidth}","p{0.14\\textwidth}", "p{0.14\\textwidth}", "p{0.14\\textwidth}")


xtable::print.xtable(
  xtable::xtable(
    t1_brand1df,
  label = label,
    caption = caption,
    floating = FALSE,
    align = align
  ),
  type = 'latex', 
  file = file.path(results_dir, "table1_brand_part1.tex"), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  floating = FALSE,
  add.to.row = add.to.row,
  tabular.environment = "longtable",
  hline.after = FALSE,
  format.args = list(digits = 3, big.mark = ",")
)

In [ ]:
t1_brand2 <- table1::table1(
    # Demographics
    ~ AgeInYears + 
    AgeGroup + 
    Sex + 
    Race + 
    Ethnicity  + 
    
    # Indications
    HadT2DEvidence + 
    HadOverweightOrObesity +  +
    #approved_condition + 
    
    #Brand
    #group + 

    #Absolute time of intitation
    #year + 
    Period + # 6 months periods of each year
    
    # Comborbidities
    HadAFib + 
    HadAsthma + 
    HadCKD + 
    HadCOPD + 
    HadGlaucoma + 
    HadHF + 
    HadHyperlipidemia + 
    HadHypertension + 
    HadIschemicHeartDisease + 
    HadPreviousAcuteMI + 
    HadPreviousIschemicStroke + 
    HadPreviousMDD + 
    HadOsteoporosis+ 
    HadPrevBariatricSurgery +
    
    # Previous medications
    UsedMetformin +
    UsedSGLT2 + 
    UsedDPP4 + 
    UsedSulfonylurea +  
    UsedInsulin + 
    UsedOrlistat + 
    UsedPhentermineTopiramate +  
    bmi +
    # Baseline Weight 
    weightInPounds +
    # Baseline A1c 
    a1c
    | brand_inferred,
    data = df[df$group_short %in% c("Dulaglutide", "Tirzepatide", "Other"),],
    render = rndr,
    render.continuous=my.render.cont, 
    render.categorical=my.render.cat,
    overall = F

)
IRdisplay::display_html(t1_brand2)
write.csv(as.data.frame(t1_brand2), file.path(results_dir, "table1_brand_part2.csv"))
t1_brand2df <- as.data.frame(t1_brand2)

caption_this_one <-  'Characteristics of patients first prescribed a GLP-1 RA, by brand for Dulaglutide, Tirzepatide, and Other.'
caption <- paste0(caption_this_one,"\\", caption_all)
label <- 'tab:table_1_brand_part2'

add.to.row <- list()
add.to.row$pos <- list()
add.to.row$pos[[1]] <- c(0)
add.to.row$command <- 
  paste0(
    "\\hline \n",
    "\\endfirsthead \n",
    "\\multicolumn{7}{p{\\textwidth}}{{ \\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\ \n",
    "\\hline Feature & Trulicity & Mounjaro & Bydureon & Byetta & Soliqua & Unknown\\\\ \\hline \n",
    "\\endhead \n",
    "\\hline \\multicolumn{7}{p{\\textwidth}}{{Continued on next page}} \\\\ \\hline \n",
    "\\endfoot \n",
    "\\hline \n",
    "\\endlastfoot \n"
  )


align <- c('', "p{0.16\\textwidth}","p{0.14\\textwidth}", "p{0.14\\textwidth}", "p{0.14\\textwidth}","p{0.14\\textwidth}", "p{0.14\\textwidth}", "p{0.14\\textwidth}")


xtable::print.xtable(
  xtable::xtable(
    t1_brand2df,
  label = label,
    caption = caption,
    floating = FALSE,
    align = align
  ),
  type = 'latex', 
  file = file.path(results_dir, "table1_brand_part2.tex"), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  floating = FALSE,
  add.to.row = add.to.row,
  tabular.environment = "longtable",
  hline.after = FALSE,
  format.args = list(digits = 3, big.mark = ",")
)

### Post May 2022 (after tirzepatide approval)

In [ ]:
## Ozempic post 2022 vs tirzepatide post 2022

post_May2022 <- df |> filter(as.Date(IndexYearMonth) > as.Date('2022-05-01'))

t1  <- table1::table1(
    # Demographics
    ~ AgeInYears + 
    AgeGroup + 
    Sex + 
    Race + 
    Ethnicity  + 
    
    # Indications
    HadT2DEvidence + 
    HadOverweightOrObesity +  +
    approved_condition + 
    
    #Brand
    brand_inferred + 

    #Absolute time of intitation
    #year + 
    Period + # 6 months periods of each year
    
    # Comborbidities
    HadAFib + 
    HadAsthma + 
    HadCKD + 
    HadCOPD + 
    HadGlaucoma + 
    HadHF + 
    HadHyperlipidemia + 
    HadHypertension + 
    HadIschemicHeartDisease + 
    HadPreviousAcuteMI + 
    HadPreviousIschemicStroke + 
    HadPreviousMDD + 
    HadOsteoporosis+ 
    HadPrevBariatricSurgery +
    
    # Previous medications
    UsedMetformin +
    UsedSGLT2 + 
    UsedDPP4 + 
    UsedSulfonylurea +  
    UsedInsulin + 
    UsedOrlistat + 
    UsedPhentermineTopiramate +  
    bmi +
    # Baseline Weight 
    weightInPounds +
    # Baseline A1c 
    a1c
    | group_short,
    data = post_May2022,
    render = rndr,
    render.continuous=my.render.cont, 
    render.categorical=my.render.cat
)
IRdisplay::display_html(t1)
write.csv(as.data.frame(t1), file.path(results_dir, "table1_generic_postMay22.csv"))
t1_genericdf <- as.data.frame(t1)

caption_this_one <-  'Characteristics of patients first prescribed a GLP-1 RA in May 2022 or later, by specific medication Other includes the two smallest groups: Exenatide and Lixisenatide.'
caption <- paste0(caption_this_one,"\\", caption_all)
label <- 'tab:table_1_generic_postMay22'

add.to.row <- list()
add.to.row$pos <- list()
add.to.row$pos[[1]] <- c(0)
add.to.row$command <- 
  paste0(
    "\\hline \n",
    "\\endfirsthead \n",
    "\\multicolumn{7}{p{\\textwidth}}{{ \\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\ \n",
    "\\hline Feature & Semaglutide & Dulaglutide & Liraglutide & Tirzepatide & Other & Overall \\\\ \\hline \n",
    "\\endhead \n",
    "\\hline \\multicolumn{7}{p{\\textwidth}}{{Continued on next page}} \\\\ \\hline \n",
    "\\endfoot \n",
    "\\hline \n",
    "\\endlastfoot \n"
  )

align <- c('', "p{0.2\\textwidth}","p{0.13\\textwidth}", "p{0.13\\textwidth}", "p{0.13\\textwidth}","p{0.13\\textwidth}","p{0.13\\textwidth}","p{0.14\\textwidth}")


xtable::print.xtable(
  xtable::xtable(
    t1_genericdf,
  label = label,
    caption = caption,
    floating = FALSE,
    align = align
  ),
  type = 'latex', 
  file = file.path(results_dir, "table1_generic_postMay22.tex"), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  floating = FALSE,
  add.to.row = add.to.row,
  tabular.environment = "longtable",
  hline.after = FALSE,
  format.args = list(digits = 3, big.mark = ",")
)

In [ ]:
## Ozempic post 2022 vs tirzepatide post 2022

t1  <- table1::table1(
    # Demographics
    ~ AgeInYears + 
    AgeGroup + 
    Sex + 
    Race + 
    Ethnicity  + 
    
    # Indications
    HadT2DEvidence + 
    HadOverweightOrObesity +  +
  #  approved_condition + 
    
    #Brand
#    brand_inferred + 

    #Absolute time of intitation
    #year + 
    Period + # 6 months periods of each year
    
    # Comborbidities
    HadAFib + 
    HadAsthma + 
    HadCKD + 
    HadCOPD + 
    HadGlaucoma + 
    HadHF + 
    HadHyperlipidemia + 
    HadHypertension + 
    HadIschemicHeartDisease + 
    HadPreviousAcuteMI + 
    HadPreviousIschemicStroke + 
    HadPreviousMDD + 
    HadOsteoporosis+ 
    HadPrevBariatricSurgery +
    
    # Previous medications
    UsedMetformin +
    UsedSGLT2 + 
    UsedDPP4 + 
    UsedSulfonylurea +  
    UsedInsulin + 
    UsedOrlistat + 
    UsedPhentermineTopiramate + 
    bmi +
    # Baseline Weight 
    weightInPounds +
    # Baseline A1c 
    a1c
    | brand_inferred,
    data = post_May2022[post_May2022$brand_inferred == c("Ozempic", "Wegovy", "Mounjaro"),],
    render = rndr,
    render.continuous=my.render.cont, 
    render.categorical=my.render.cat,
    overall = F
)
IRdisplay::display_html(t1)
write.csv(as.data.frame(t1), file.path(results_dir, "table1_brand_postMay22.csv"))
t1_df <- as.data.frame(t1)

caption_this_one <-  'Characteristics of patients first prescribed a GLP-1 RA in May 2022 or later, for Ozempic, Wegovy, and Mounjaro.'
caption <- paste0(caption_this_one,"\\", caption_all)
label <- 'tab:table_1_brand_postMay22'

add.to.row <- list()
add.to.row$pos <- list()
add.to.row$pos[[1]] <- c(0)
add.to.row$command <- 
  paste0(
    "\\hline \n",
    "\\endfirsthead \n",
    "\\multicolumn{4}{p{\\textwidth}}{{ \\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\ \n",
    "\\hline Feature & Ozempic & Wegovy & Moujaro\\\\ \\hline \n",
    "\\endhead \n",
    "\\hline \\multicolumn{4}{p{\\textwidth}}{{Continued on next page}} \\\\ \\hline \n",
    "\\endfoot \n",
    "\\hline \n",
    "\\endlastfoot \n"
  )

align <- c('', "p{0.3\\textwidth}","p{0.2\\textwidth}", "p{0.2\\textwidth}", "p{0.2\\textwidth}")


xtable::print.xtable(
  xtable::xtable(
    t1_df,
  label = label,
    caption = caption,
    floating = FALSE,
    align = align
  ),
  type = 'latex', 
  file = file.path(results_dir, "table1_brand_postMay22.tex"), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  floating = FALSE,
  add.to.row = add.to.row,
  tabular.environment = "longtable",
  hline.after = FALSE,
  format.args = list(digits = 3, big.mark = ",")
)

# Plots

## FIgure 2: Rx Over time, by (A) Specific Medication, (B) Brand, and (C) FDA Labelled Use

### Specific Medication

In [ ]:
initialize_theme_truveta(figsize = c(12, 6))

p_generic <- df |>
    ggplot(aes(x = as.Date(IndexYearMonth))) +
    geom_point(aes(color = group, shape = group), stat = "count", size = 1.5) +
    geom_line(aes(color = group), stat = "count") +
    ggplot2::scale_color_manual(values = truveta.research::official_color_palette) +
    ggplot2::scale_x_date(date_labels = "%b %Y", date_breaks = "3 month",
                         limits = as.Date(c("2018-01-01", "2023-06-01")),
                        expand = c(0,0)) +
    ggplot2::scale_size_manual(values = c(.5,1.5)) +
    ggplot2::guides(size = "none") +
    ggplot2::labs(x = "\nTime",
        y = "Number of First-Time GLP-1 RA Prescriptions",
        title = "First-Time GLP-1 RA Prescriptions\n By Specific Medication") + 
    theme_truveta() +
    ggplot2::theme(axis.text.x = ggplot2::element_text(angle = 30,hjust=1),
                  axis.text = ggplot2::element_text(size = 10),
                  legend.text = ggplot2::element_text(size = 8),
                  legend.title =element_blank(),
                  plot.title = ggplot2::element_text(size = 12, hjust = .5),
                  legend.position = c(0.05, 0.75)
                  ) +
 guides(col = guide_legend(nrow = 3))
p_generic
ggsave(file.path(results_dir, "RxOverTime_substance.png"),p_generic)



### Brand

In [ ]:
s <- c(16, 17, 15, 3,   7, 8, 17,  4, 10, 18, 16, 4)

initialize_theme_truveta(figsize = c(12, 6))
p_brand <- df |>
    ggplot(aes(x = as.Date(IndexYearMonth))) +
    geom_point(aes(color = brand_inferred, fill = brand_inferred, shape = brand_inferred), stat = "count", size = 1.5) +
    scale_shape_manual(values=s)+
    geom_line(aes(color = brand_inferred), stat = "count") +
    ggplot2::scale_color_manual(values = truveta.research::official_color_palette) +
    ggplot2::scale_x_date(date_labels = "%b %Y", 
                          date_breaks = "3 month",
                         limits = as.Date(c("2018-01-01", "2023-06-01")),
                        expand = c(0,0)) +
    ggplot2::guides(size = "none") +
    ggplot2::labs(x = "\nTime",
        y = "Number of First-Time GLP-1 RA Prescriptions",
        title = "First-Time GLP-1 RA Prescriptions\n By Brand") + 
    theme_truveta() +
    ggplot2::theme(axis.text.x = ggplot2::element_text(angle = 30,hjust=1),
                  axis.text = ggplot2::element_text(size = 10),
                  legend.text = ggplot2::element_text(size = 8),
                  legend.title =element_blank(),
                  plot.title = ggplot2::element_text(size = 12, hjust = .5),
                  legend.position = c(0.05, 0.75)
                  ) +
 guides(col = guide_legend(nrow = 4),
       shape = guide_legend(nrow = 4),
       fill = guide_legend(nrow = 4))
  
p_brand
ggsave(file.path(results_dir, "RxOverTime_brand.png"), p_brand)

### FDA Label

In [ ]:
initialize_theme_truveta(figsize = c(12,6))

p_label <- df |>
    ggplot(aes(x = as.Date(IndexYearMonth), color = approved_condition)) +
    geom_point(aes(color = approved_condition, shape = approved_condition), stat = "count", size = 1.5) +
    geom_line(stat = "count") +
    ggplot2::scale_color_manual(values = truveta.research::official_color_palette) +
    ggplot2::scale_x_date(date_labels = "%b %Y", date_breaks = "3 month",
                         limits = as.Date(c("2018-01-01", "2023-06-01")),
                        expand = c(0,0)) +
    ggplot2::scale_size_manual(values = c(.5,1.5)) +
    ggplot2::guides(size = "none") +
    ggplot2::labs(x = "\nTime",
        y = "Number of First-Time GLP-1 RA Prescriptions",
        title = "First-Time GLP-1 RA Prescriptions \n By FDA-Labelled Condition") + 
    theme_truveta() +
    ggplot2::theme(axis.text.x = ggplot2::element_text(angle = 30,hjust=1),
                  axis.text = ggplot2::element_text(size = 10),
                  legend.text = ggplot2::element_text(size = 8),
                  legend.title =element_blank(),
                  plot.title = ggplot2::element_text(size = 12, hjust = .5),
                  legend.position = c(0.05, 0.75)) 
p_label
ggsave(file.path(results_dir, "RxOverTime_FDALabel.png"),p_label)

In [ ]:
# Combined into a multi-panel plot
pa <- p_generic + ggplot2::labs(title = "(A) First-Time GLP-1 RA Prescriptions\n By Specific Medication", y = "")
pb <- p_brand +   ggplot2::labs(title = "(B) First-Time GLP-1 RA Prescriptions\n By Brand")
pc <- p_label +   ggplot2::labs(title = "(C) First-Time GLP-1 RA Prescriptions\n By FDA Labelled Use", y = "")


combined <- grid.arrange(pa, pb, pc)
ggsave(file=file.path(results_dir, "combined.png"), combined, height = 10, width = 8)
ggsave(file=file.path(results_dir, "combined.svg"), combined, height = 10, width = 8)

## Figure 3: On vs Off Label Use, by T2D-Labelled vs. Obesity Labelled

In [ ]:
names(known_label)

In [ ]:
initialize_theme_truveta(figsize = c(9,12))

known_label |>
    mutate(OnLabel = factor(OnLabel, levels = c(TRUE, FALSE), labels = c('On-label', 'Off-label'))) |>
    filter(group %in% c('Semaglutide', 'Liraglutide', 'Dulaglutide', 'Tirzepatide')) |>
    group_by(IndexYearMonth, group, OnLabel, approved_condition) |> 
    tally() |> 
    ungroup() |>
    ggplot(aes(x = as.Date(IndexYearMonth), y = n)) +
    facet_wrap(~approved_condition, ncol = 1) +
    geom_line( aes(color = group, linetype = OnLabel)) +
 # geom_point(aes(color = group, shape = group, group = onlabel), size = 1) +
    ggplot2::scale_color_manual(values = truveta.research::official_color_palette) +
    ggplot2::scale_x_date(date_labels = "%b %Y", date_breaks = "3 month",
                         limits = as.Date(c("2018-01-01", "2023-06-01")),
                        expand = c(0,0)) +
    ggplot2::scale_size_manual(values = c(.5,1.5)) +
    ggplot2::guides(size = "none") +
    ggplot2::labs(x = "\nTime",
        y = "Number of First-Time GLP-1 RA Prescriptions",
        title = "First-Time GLP-1 RA Prescriptions by Medication \n By Labelled Indication and On- vs -Off Label Use") + 
    theme_truveta() +
    ggplot2::theme(axis.text.x = ggplot2::element_text(angle = 30, hjust=1),
                  axis.text = ggplot2::element_text(size = 10),
                  legend.text = ggplot2::element_text(size = 10),
                  legend.title =element_blank(),
                  plot.title = ggplot2::element_text(size = 12, hjust = .5)) +
 guides(col = guide_legend(nrow = 2),
       linetype = guide_legend(nrow =2))

ggsave(file.path(results_dir, "RxOverTime_OnLabel_keygroups.png"),last_plot())
ggsave(file.path(results_dir, "RxOverTime_OnLabel_keygroups.svg"),last_plot())

#write_ggplot(last_plot(), file.path(results_dir, "RxOverTime_OnLabel_keygroups.png"))

## Reference populations

In [ ]:
person_ts <- arrow::read_parquet(file.path(data_dir, "persons_with_encounter_timeseries.parquet"))
person_uc_ts <- arrow::read_parquet(file.path(data_dir, "persons_with_encounter_UC_timeseries.parquet"))

In [ ]:
unique(person_ts$aggregate)

In [ ]:
# Total Encounters over time
person_ts |> 
    mutate(aggregate = factor(aggregate, 
                             levels = c('agg_PersonEnc', 'agg_PersonEncAdult', 'agg_PersonEncAdultOO',  'agg_PersonEncAdultT2D' ),
                            labels = c('All Patients', 'Adults',  'Adults with Overweight or Obesity', 'Adults with T2D')))|>
ggplot( aes(x = as.Date(Date), y = Value, color = aggregate, linetype = aggregate)) +
    geom_line() +
    ggplot2::scale_color_manual(values = truveta.research::official_color_palette) +
    ggplot2::scale_linetype_manual(values = c( "solid", "longdash", "dotdash",  "twodash")) +
    ggplot2::scale_x_date(date_labels = "%b %Y", date_breaks = "3 month",
                         limits = as.Date(c("2018-01-01", "2023-07-01")),
                        expand = c(0,0)) +
    ggplot2::scale_size_manual(values = c(.5,1.5)) +
    ggplot2::guides(size = "none") +
    ggplot2::labs(x = "\nTime",
        y = "Persons",
        title = "Total Patients with Encounters \nby Month") + 
    theme_truveta(figsize = c(12,7)) +
  	scale_y_continuous(limits = c(0, 10000000), labels = label_number(suffix = " M", scale = 1e-6)) +
    ggplot2::theme(axis.text.x = ggplot2::element_text(angle = 30, hjust=1),
                  axis.text = ggplot2::element_text(size = 12),
                  legend.text = ggplot2::element_text(size = 12),
                  legend.title =element_blank(),
                  plot.title = ggplot2::element_text(size = 14, hjust = .5),
                  legend.position = c(0.01, 0.85)) +
 guides(col = guide_legend(nrow = 4))

ggsave(file.path(results_dir, "Denominator_PersonWithEncounter.png"),last_plot())

In [ ]:
# Patients with usual care encounters
person_uc_ts |> 
    mutate(aggregate = factor(aggregate, 
                             levels = c('agg_PersonEncAdultUc',  'agg_PersonEncAdultOOUc','agg_PersonEncAdultT2DUc'),
                            labels = c('Adults',  'Adults with Overweight or Obesity', 'Adults with T2D')))|>
ggplot( aes(x = as.Date(Date), y = Value, color = aggregate, linetype = aggregate)) +
    geom_line() +
    ggplot2::scale_color_manual(values = truveta.research::official_color_palette[2:4]) +
    ggplot2::scale_x_date(date_labels = "%b %Y", date_breaks = "3 month",
                         limits = as.Date(c("2018-01-01", "2023-06-01")),
                        expand = c(0,0)) +
    ggplot2::scale_size_manual(values = c(.5,1.5)) +
    ggplot2::guides(size = "none") +
    ggplot2::labs(x = "\nTime",
        y = "Persons",
        title = "Total Usual Care Patients with Encounters\n By Month",
        caption = "A usual care patient is defined as having a usual care outpatient visit in each consecutive 6-month period before the encounter.") + 
    theme_truveta(figsize = c(12,7)) +
  	scale_y_continuous(limits = c(0, 5000000), labels = label_number(suffix = " M", scale = 1e-6)) +
    ggplot2::theme(axis.text.x = ggplot2::element_text(angle = 30, hjust=1),
                  axis.text = ggplot2::element_text(size = 12),
                  legend.text = ggplot2::element_text(size = 12),
                  legend.title =element_blank(),
                  plot.title = ggplot2::element_text(size = 14, hjust = .5),
                  legend.position = c(0.01, 0.85)) +
 guides(col = guide_legend(nrow = 4))


ggsave(file.path(results_dir, "Denominator_PersonWithEncounterAndUsualCare.png"),last_plot())

In [ ]:
funnel_denominator <- arrow::read_parquet(file.path(data_dir, "funnel_denominator.parquet"))

In [ ]:
## Re-create parallel funnel for entire population, for reference, using aggregates

funnel_denominator <- arrow::read_parquet(file.path(data_dir, "funnel_denominator.parquet"))
#funnel_denominator          

by_condition_any <-
    bind_rows( 
           df |> 
                mutate(group = "Adults",
                       Step = "Prescribed GLP-1 RA") |>
                group_by(group, Step) |>
                summarize(Value = n()),
            df[df$HadT2DEvidence,] |> 
                mutate(group = "Any T2D",
                       Step = "Prescribed GLP-1 RA") |>
                group_by(group, Step) |>
                summarize(Value = n()),
            df[df$HadOverweightOrObesity,] |> 
                mutate(group = "Any Overweight or Obesity",
                       Step = "Prescribed GLP-1 RA")|>
                group_by(group, Step) |>
                summarize(Value = n())
        )

by_condition_exclusive <- df |> 
    mutate(group = case_when(
               HadT2DEvidence &  HadOverweightOrObesity ~ "Both",
               HadT2DEvidence &  !HadOverweightOrObesity ~ "T2D Only",
               !HadT2DEvidence &  HadOverweightOrObesity ~ "Overweight or Obesity Only",
               !HadT2DEvidence &  !HadOverweightOrObesity ~ "Neither"),
           Step = "Prescribed GLP-1 RA") |>
 group_by(group, Step) |>
 summarize(Value = n())


funnel_denominator <- bind_rows(funnel_denominator, by_condition_any, by_condition_exclusive )

In [ ]:
# On Label
by_condition_any_ol <-
    bind_rows( 
            df[df$OnLabel,] |> 
                mutate(group = "Adults",
                       Step = "On-label") |>
                group_by(group, Step) |>
                summarize(Value = n()),
            df[df$HadT2DEvidence & df$OnLabel,] |> 
                mutate(group = "Any T2D",
                       Step = "On-label") |>
                group_by(group, Step) |>
                summarize(Value = n()),
            df[df$HadOverweightOrObesity & df$OnLabel,] |> 
                mutate(group = "Any Overweight or Obesity",
                       Step = "On-label")|>
                group_by(group, Step) |>
                summarize(Value = n())
        )

by_condition_exclusive_ol <- df |> 
    filter(OnLabel) |>
    mutate(group = case_when(
               HadT2DEvidence &  HadOverweightOrObesity ~ "Both",
               HadT2DEvidence &  !HadOverweightOrObesity ~ "T2D Only",
               !HadT2DEvidence &  HadOverweightOrObesity ~ "Overweight or Obesity Only",
               !HadT2DEvidence &  !HadOverweightOrObesity ~ "Neither"),
           Step = "On-label") |>
 group_by(group, Step) |>
 summarize(Value = n())

funnel_denominator <- bind_rows(funnel_denominator, by_condition_any_ol, by_condition_exclusive_ol) 

In [ ]:
funnel_denominator <- funnel_denominator|> 
    mutate(Stepfmt = factor(Step, 
                        levels = c('All','Study Criteria','Prescribed GLP-1 RA','On-label'),
                        labels = c('Adults with encounter','Usual care & no exclusions','Prescribed GLP-1 RA','On-label')))

In [ ]:
plt <- funnel_denominator |>
    filter(group %in% c('Adults','Any T2D','Any Overweight or Obesity')) |>
    mutate(group = factor(group, 
                          levels = c('Adults','Any T2D','Any Overweight or Obesity'),
                          labels = c('Adults','Adults with T2D','Adults with Overweight or Obesity'))) |>
ggplot( aes(x = Stepfmt, y = Value, fill = group)) +
  #  facet_wrap(~group, scales = "free") +
    geom_bar(stat = "identity", position = "dodge") + 
    geom_text(aes(label=paste0(format(round(Value/1000000,1),nsmall = 1), " M")), size = 3, position=position_dodge(width=0.9), vjust=-0.25) +
    ggplot2::scale_fill_manual(values = truveta.research::official_color_palette) +
    ggplot2::labs(
        x = "",
        y = "Patients \n",
        title = "Patient Volumes by Step") + 
    theme_truveta(figsize = c(14,8)) +
  	scale_y_continuous(labels = label_number(suffix = " M", scale = 1e-6)) +
    scale_x_discrete(labels = function(x) str_wrap(x, width = 10)) +
    ggplot2::theme(
                #axis.text.x = ggplot2::element_text(angle = 30,hjust=1),
                  axis.text = ggplot2::element_text(size = 10),
                  legend.text = ggplot2::element_text(size = 10),
                  legend.title =element_blank(),
                  plot.title = ggplot2::element_text(size = 12, hjust = .5),
                 # legend.position = c(0.6, 0.7)
                  ) +
 guides(col = guide_legend(nrow = 3))
plt
ggsave(file.path(results_dir, "Funnel.png"),plt)

In [ ]:
plt <- funnel_denominator |>
    filter(group %in% c( 'Overweight or Obesity Only','T2D Only','Both', 'Neither')) |>
    mutate(category = factor(group,
                            levels = c('T2D Only', 'Overweight or Obesity Only','Both', 'Neither'))) |>
    group_by(Stepfmt) |>
    mutate(p = Value/sum(Value)) |>
ggplot( aes(x = Stepfmt, y = p, fill = category)) +
  #  facet_wrap(~group, scales = "free") +
    geom_bar(stat = "identity", position = "stack") + 
    geom_text(aes(label=paste0(format(round(Value/1000000,1),nsmall = 1), " M")), size = 4, position = position_stack(vjust = 0.5), color = "grey95") +
    ggplot2::scale_fill_manual(values = truveta.research::official_color_palette) +
    ggplot2::labs(
        x = "",
        y = "Percent of Total \n",
        title = "Proportion by Step") + 
    theme_truveta(figsize = c(14,8)) +
  scale_y_continuous(labels = scales::percent) +
    scale_x_discrete(labels = function(x) str_wrap(x, width = 10)) +
    ggplot2::theme(
                #axis.text.x = ggplot2::element_text(angle = 30,hjust=1),
                  axis.text = ggplot2::element_text(size = 10),
                  legend.text = ggplot2::element_text(size = 10),
                  legend.title =element_blank(),
                  plot.title = ggplot2::element_text(size = 12, hjust = .5)
                  ) +
 guides(col = guide_legend(nrow = 3))
plt
ggsave(file.path(results_dir, "FunnelProportion.png"),plt)